##### Авторские права 2020 Авторы TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Введение в градиенты и автоматическое дифференцирование

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/autodiff"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Посмотреть на TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Запустить в Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Посмотреть источник на GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/autodiff.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Скачать блокнот</a></td>
</table>

## Автоматическое дифференцирование и градиенты

[Автоматическое дифференцирование](https://en.wikipedia.org/wiki/Automatic_differentiation) полезно для реализации алгоритмов машинного обучения, таких как [обратное распространение](https://en.wikipedia.org/wiki/Backpropagation) для обучения нейронных сетей.

В этом руководстве мы обсудим способы вычисления градиентов с помощью TensorFlow, особенно в стремительном исполнении.

## Настроить

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

## Вычисление градиентов

Для автоматической дифференциации TensorFlow необходимо запомнить, какие операции происходят в каком порядке во время *прямого* прохода. Затем во время *обратного прохода* TensorFlow перебирает этот список операций в обратном порядке для вычисления градиентов.

## Градиентные ленты

TensorFlow предоставляет API [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) для автоматической дифференциации; то есть вычисление градиента вычисления относительно некоторых входных данных, обычно `tf.Variable` s. TensorFlow «записывает» соответствующие операции, выполненные в контексте `tf.GradientTape` на «ленту». Затем TensorFlow использует эту ленту для вычисления градиентов «записанных» вычислений с использованием [дифференцирования в обратном режиме](https://en.wikipedia.org/wiki/Automatic_differentiation) .

Вот простой пример:

In [ ]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
  y = x**2

После того, как вы записали некоторые операции, используйте `GradientTape.gradient(target, sources)` чтобы вычислить градиент некоторой цели (часто потери) относительно некоторого источника (часто переменных модели).

In [ ]:
# dy = 2x * dx
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

В приведенном выше примере используются скаляры, но `tf.GradientTape` работает так же легко на любом тензоре:

In [ ]:
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)

Чтобы получить градиент `y` относительно обеих переменных, вы можете передать оба источника в метод `gradient` . Лента обладает гибкостью в отношении передачи источников и будет принимать любую вложенную комбинацию списков или словарей и возвращать градиент, структурированный таким же образом (см. `tf.nest` ).

In [ ]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

Градиент по отношению к каждому источнику имеет форму источника:

In [ ]:
print(w.shape)
print(dl_dw.shape)

Вот снова вычисление градиента, на этот раз передавая словарь переменных:

In [ ]:
my_vars = {
    'w': tf.Variable(tf.random.normal((3, 2)), name='w'),
    'b': tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
}

grad = tape.gradient(loss, my_vars)
grad['b']

## Градиенты по отношению к модели

Распространено собирать `tf.Variables` в `tf.Module` или в один из его подклассов ( `layers.Layer` , `keras.Model` ) для [проверки](checkpoint.ipynb) и [экспорта](saved_model.ipynb) .

В большинстве случаев вы захотите рассчитать градиенты относительно обучаемых переменных модели. Поскольку все подклассы `tf.Module` агрегируют свои переменные в свойстве `Module.trainable_variables` , вы можете рассчитать эти градиенты в нескольких строках кода: 

In [ ]:
layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
  # Forward pass
  y = layer(x)
  loss = tf.reduce_mean(y**2)

# Calculate gradients with respect to every trainable variable
grad = tape.gradient(loss, layer.trainable_variables)

In [ ]:
for var, g in zip(layer.trainable_variables, grad):
  print(f'{var.name}, shape: {g.shape}')

<a id="watches"></a>

## Управление тем, что смотрит лента

Поведение по умолчанию - записывать все операции после доступа к обучаемому `tf.Variable` . Причины этого:

- Лента должна знать, какие операции записывать в прямом проходе, чтобы рассчитать градиенты в обратном проходе.
- Лента содержит ссылки на промежуточные выходы, поэтому вы не хотите записывать ненужные операции.
- Наиболее распространенный вариант использования включает вычисление градиента потерь по всем обучаемым переменным модели.

Например, следующее не может рассчитать градиент, потому что `tf.Tensor` не «отслеживается» по умолчанию, а `tf.Variable` не обучаем:

In [ ]:
# A trainable variable
x0 = tf.Variable(3.0, name='x0')
# Not trainable
x1 = tf.Variable(3.0, name='x1', trainable=False)
# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name='x2') + 1.0
# Not a variable
x3 = tf.constant(3.0, name='x3')

with tf.GradientTape() as tape:
  y = (x0**2) + (x1**2) + (x2**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
  print(g)

Вы можете перечислить переменные, отслеживаемые лентой, используя метод `GradientTape.watched_variables` :

In [ ]:
[var.name for var in tape.watched_variables()]

`tf.GradientTape` предоставляет хуки, которые дают пользователю контроль над тем, что просматривается или не отслеживается.

Чтобы записать градиенты относительно `tf.Tensor` , вам нужно вызвать `GradientTape.watch(x)` :

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
  tape.watch(x)
  y = x**2

# dy = 2x * dx
dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())

И наоборот, чтобы отключить поведение по умолчанию просмотра всех `tf.Variables` , установите `watch_accessed_variables=False` при создании ленты градиента. Этот расчет использует две переменные, но связывает градиент только для одной из переменных:

In [ ]:
x0 = tf.Variable(0.0)
x1 = tf.Variable(10.0)

with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(x1)
  y0 = tf.math.sin(x0)
  y1 = tf.nn.softplus(x1)
  y = y0 + y1
  ys = tf.reduce_sum(y)

Поскольку `GradientTape.watch` не был вызван в `x0` , градиент не вычисляется относительно него:

In [ ]:
# dy = 2x * dx
grad = tape.gradient(ys, {'x0': x0, 'x1': x1})

print('dy/dx0:', grad['x0'])
print('dy/dx1:', grad['x1'].numpy())

## Промежуточные результаты

Вы также можете запросить градиенты вывода относительно промежуточных значений, вычисленных в контексте `tf.GradientTape` .

In [ ]:
x = tf.constant(3.0)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = x * x
  z = y * y

# Use the tape to compute the gradient of z with respect to the
# intermediate value y.
# dz_dx = 2 * y, where y = x ** 2
print(tape.gradient(z, y).numpy())

By default, the resources held by a `GradientTape` are released as soon as `GradientTape.gradient()` method is called. To compute multiple gradients over the same computation, create a `persistent` gradient tape. This allows multiple calls to the `gradient()` method as resources are released when the tape object is garbage collected. For example:

In [ ]:
x = tf.constant([1, 3.0])
with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  y = x * x
  z = y * y

print(tape.gradient(z, x).numpy())  # 108.0 (4 * x**3 at x = 3)
print(tape.gradient(y, x).numpy())  # 6.0 (2 * x)

In [ ]:
del tape   # Drop the reference to the tape

## Примечания по производительности

- Есть небольшие накладные расходы, связанные с выполнением операций в контексте градиентной ленты. Для большинства стремящихся к исполнению это не будет заметной стоимостью, но вы все равно должны использовать контекст ленты только в тех областях, где это требуется.

- Градиентные ленты используют память для хранения промежуточных результатов, включая входы и выходы, для использования во время обратного прохода.

    Для эффективности некоторым операциям (например, `ReLU` ) не нужно сохранять промежуточные результаты, и они сокращаются во время прямого прохода. Однако, если вы используете `persistent=True` на вашей ленте, *ничего не удаляется,* и ваше пиковое использование памяти будет выше.

## Градиенты нескалярных целей

Градиент - это в основном операция на скаляре.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient(y0, x).numpy())
print(tape.gradient(y1, x).numpy())

Таким образом, если вы запрашиваете градиент нескольких целей, результат для каждого источника:

- Градиент суммы целей или эквивалентно
- Сумма градиентов каждой цели.

In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape() as tape:
  y0 = x**2
  y1 = 1 / x

print(tape.gradient({'y0': y0, 'y1': y1}, x).numpy())

Аналогично, если цель (и) не являются скалярными, вычисляется градиент суммы:

In [ ]:
x = tf.Variable(2.)

with tf.GradientTape() as tape:
  y = x * [3., 4.]

print(tape.gradient(y, x).numpy())

Это упрощает выбор градиента суммы совокупных потерь или градиента суммы поэлементного расчета убытков.

Если вам нужен отдельный градиент для каждого предмета, смотрите [Якобиан](advanced_autodiff.ipynb#jacobians) .

В некоторых случаях вы можете пропустить якобиана. Для поэлементного вычисления градиент суммы дает производную каждого элемента по отношению к его входному элементу, поскольку каждый элемент независим:

In [ ]:
x = tf.linspace(-10.0, 10.0, 200+1)

with tf.GradientTape() as tape:
  tape.watch(x)
  y = tf.nn.sigmoid(x)

dy_dx = tape.gradient(y, x)

In [ ]:
plt.plot(x, y, label='y')
plt.plot(x, dy_dx, label='dy/dx')
plt.legend()
_ = plt.xlabel('x')

## Контроль потока

Поскольку ленты записывают операции по мере их выполнения, поток управления Python (с использованием `if` s и `while` s) обрабатывается естественным образом.

Здесь разные переменные используются в каждой ветви `if` . Градиент соединяется только с той переменной, которая была использована:

In [ ]:
x = tf.constant(1.0)

v0 = tf.Variable(2.0)
v1 = tf.Variable(2.0)

with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  if x > 0.0:
    result = v0
  else:
    result = v1**2 

dv0, dv1 = tape.gradient(result, [v0, v1])

print(dv0)
print(dv1)

Просто помните, что сами управляющие операторы не дифференцируемы, поэтому они невидимы для оптимизаторов на основе градиента.

В зависимости от значения `x` в приведенном выше примере лента либо записывает `result = v0` либо `result = v1**2` . Градиент по отношению к `x` всегда равен `None` .

In [ ]:
dx = tape.gradient(result, x)

print(dx)

## Получение градиента `None`

Когда цель не связана с источником, вы получите градиент `None` .


In [ ]:
x = tf.Variable(2.)
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y * y
print(tape.gradient(z, x))

Здесь `z` , очевидно, не связан с `x` , но есть несколько менее очевидных способов отсоединения градиента.

### 1. Заменил переменную на тензор.

В разделе [«Управление тем, что смотрит лента»,](#watches) вы увидели, что лента будет автоматически наблюдать `tf.Variable` но не `tf.Tensor` .

Одна из распространенных ошибок заключается в непреднамеренной замене `tf.Variable` на `tf.Tensor` вместо использования `Variable.assign` для обновления `tf.Variable` . Вот пример:

In [ ]:
x = tf.Variable(2.0)

for epoch in range(2):
  with tf.GradientTape() as tape:
    y = x+1

  print(type(x).__name__, ":", tape.gradient(y, x))
  x = x + 1   # This should be `x.assign_add(1)`

### 2. Делали расчеты вне TensorFlow

Лента не может записать путь градиента, если расчет выходит из TensorFlow. Например:

In [ ]:
x = tf.Variable([[1.0, 2.0],
                 [3.0, 4.0]], dtype=tf.float32)

with tf.GradientTape() as tape:
  x2 = x**2

  # This step is calculated with NumPy
  y = np.mean(x2, axis=0)

  # Like most ops, reduce_mean will cast the NumPy array to a constant tensor
  # using `tf.convert_to_tensor`.
  y = tf.reduce_mean(y, axis=0)

print(tape.gradient(y, x))

### 3. Взял градиенты через целое число или строку

Целые числа и строки не дифференцируемы. Если путь расчета использует эти типы данных, градиента не будет.

Никто не ожидает, что строки будут дифференцируемыми, но легко создать случайную константу или переменную `int` если вы не укажете `dtype` .

In [ ]:
# The x0 variable has an `int` dtype.
x = tf.Variable([[2, 2],
                 [2, 2]])

with tf.GradientTape() as tape:
  # The path to x1 is blocked by the `int` dtype here.
  y = tf.cast(x, tf.float32)
  y = tf.reduce_sum(x)

print(tape.gradient(y, x))

TensorFlow не выполняет автоматическое приведение между типами, поэтому на практике вы часто получаете ошибку типа вместо отсутствующего градиента.

### 5. Взял градиенты через объект с состоянием

Состояние останавливает градиенты. Когда вы читаете из объекта с состоянием, лента может видеть только текущее состояние, а не историю, которая к нему ведет.

`tf.Tensor` является неизменным. Вы не можете изменить тензор, как только он создан. У него есть *значение* , но нет *государства* . Все обсуждаемые до сих пор операции также не имеют состояния: вывод `tf.matmul` зависит только от его входных данных.

`tf.Variable` имеет внутреннее состояние, его значение. Когда вы используете переменную, состояние читается. Это нормально для вычисления градиента относительно переменной, но состояние переменной блокирует вычисления градиента от дальнейшего продвижения назад. Например:


In [ ]:
x0 = tf.Variable(3.0)
x1 = tf.Variable(0.0)

with tf.GradientTape() as tape:
  # Update x1 = x1 + x0.
  x1.assign_add(x0)
  # The tape starts recording from x1.
  y = x1**2   # y = (x1 + x0)**2

# This doesn't work.
print(tape.gradient(y, x0))   #dy/dx0 = 2*(x1 + x2)

Аналогично, итераторы `tf.data.Dataset` и `tf.queue` s сохраняют состояние и останавливают все градиенты на тензорах, проходящих через них.

## Градиент не зарегистрирован

Some `tf.Operation`s are **registered as being non-differentiable** and will return `None`. Others have **no gradient registered**.

The [tf.raw_ops](https://www.tensorflow.org/api_docs/python/tf/raw_ops) page shows which low-level ops have gradients registered.

Если вы попытаетесь выполнить градиент через операцию с плавающей точкой, у которой нет зарегистрированного градиента, лента выдаст ошибку, а не вернет `None` . Таким образом, вы знаете, что что-то пошло не так.

Например, функция `tf.image.adjust_contrast` оборачивает [raw_ops.AdjustContrastv2,](https://www.tensorflow.org/api_docs/python/tf/raw_ops#.AdjustContrastv2) который может иметь градиент, но градиент не реализован:


In [ ]:
image = tf.Variable([[[0.5, 0.0, 0.0]]])
delta = tf.Variable(0.1)

with tf.GradientTape() as tape:
  new_image = tf.image.adjust_contrast(image, delta)

try:
  print(tape.gradient(new_image, [image, delta]))
  assert False   # This should not happen.
except LookupError as e:
  print(f'{type(e).__name__}: {e}')


Если вам нужно провести различие между этими операциями, вам нужно либо реализовать градиент и зарегистрировать его (используя `tf.RegisterGradient` ), либо повторно реализовать функцию, используя другие операции.

## Нули вместо None

В некоторых случаях было бы удобно получить 0 вместо `None` для неподключенных градиентов. Вы можете решить, что возвращать, когда у вас есть неподключенные градиенты, используя аргумент `unconnected_gradients` :

In [ ]:
x = tf.Variable([2., 2.])
y = tf.Variable(3.)

with tf.GradientTape() as tape:
  z = y**2
print(tape.gradient(z, x, unconnected_gradients=tf.UnconnectedGradients.ZERO))